# Tratamento dos dados

Aqui está documentado o processo de tratamento aplicado aos dados coletados no [notebook 1](1-coleta.ipynb).

Os dados coletados durante o webscraping foram armazenados da forma mais crua possível. Via de regra, representam as strings que puderam ser extraídas de elementos HTML específicos, com o auxílio da biblioteca **BeautifulSoup**.

Na maioria dos casos, essas strings já possuem um dado atomizado, merecendo, talvez, algum tratamento de conversão de tipo (para números ou datetimes, por exemplo) ou padronização (como os nomes das equipes).


Em outros, os dados representam algum tipo de coleção (como no caso do local da partida, que contém o nome do estádio e, muitas vezes, também a cidade e o Estado, ou da relação de gols marcados por equipe), precisando, portanto, serem desmembrados para que cada componente receba um tratamento específico.

## Setup

Utilizaremos as bibliotecas **datetime** para parsing de datas e **re** para manipulação de strings. A biblioteca **locale** auxiliará no parsing das strings de data escritas em português brasileiro. Por fim, as bibliotecas **numpy** e **pandas** oferecem as ferramentas para o tratamento agregado dos dados e posterior exportação do resultado.

In [1]:
from datetime import datetime
import locale
import re

import numpy as np
import pandas as pd
import pyarrow

In [2]:
locale.setlocale(locale.LC_TIME, 'pt_BR')

'pt_BR'

## Carregando os dados brutos

Os dados originais foram salvos sem qualquer identificação dos campos. Por ora, utilizaremos nomes genéricos paras colunas. Depois de carregados, poderemos identificar a quais campos correspondem.

In [3]:
# o csv de dados brutos possui 16 colunas
cols = ['col%02d'%n for n in range(1, 17)]

# carregamento
raw = pd.read_csv('dados/raw-scrap.csv', names=cols)
# amostra
raw.head()

,col01,col02,col03,col04,col05,col06,col07,col08,col09,col10,col11,col12,col13,col14,col15,col16
0,campeonato-brasileiro-serie-a,2012,1,São Januário - Rio de Janeiro - RJ,Domingo,20 de Maio de 2012,18:30,2,Vasco da Gama - RJ,NaN,1.0,Grêmio - RS,NaN,NaN,0.0,0.0
1,campeonato-brasileiro-serie-a,2012,2,Pituaçu - Salvador - BA,Domingo,20 de Maio de 2012,18:30,0,Bahia - BA,NaN,0.0,Santos - SP,NaN,NaN,0.0,0.0
2,campeonato-brasileiro-serie-a,2012,3,Pacaembu - Sao Paulo - SP,Sábado,19 de Maio de 2012,18:30,1,Palmeiras - SP,NaN,1.0,Portuguesa - SP,NaN,NaN,0.0,0.0
3,campeonato-brasileiro-serie-a,2012,4,Orlando Scarpelli - Florianopolis - SC,Sábado,19 de Maio de 2012,21:00,2,Figueirense - SC,NaN,1.0,Náutico - PE,NaN,NaN,0.0,0.0
4,campeonato-brasileiro-serie-a,2012,5,Pacaembu - Sao Paulo - SP,Domingo,20 de Maio de 2012,16:00,0,Corinthians - SP,NaN,1.0,Fluminense - RJ,NaN,NaN,0.0,0.0


A amostra acima indica a existência dos seguintes campos em cada coluna:
- col01: competição
- col02: temporada
- col03: id_partida
- col04: estádio [, cidade, uf]
- col05: dia da semana
- col06: data por extenso
- col07: hora
- col08: placar do tempo normal para a equipe mandante
- col09: nome da equipe mandante
- col10: jogadores marcaram gols a favor da equipe mandante
- col11: placar do tempo normal para a equipe visitante
- col12: nome da equipe visitante
- col13: jogadores que marcaram gols a favor da equipe visitante
- col14: jogadores que marcaram gol contra a própria equipe
- col15: placar de disputa de pênaltis para a equipe mandante
- col16: placar de disputa de pênaltis para a equipe visitante

Vejamos, agora, algumas informações preliminares sobre o formato, os tipos de dados e possíveis faltantes do dataset:

In [4]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5214 entries, 0 to 5213
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   col01   5214 non-null   object 
 1   col02   5214 non-null   int64  
 2   col03   5214 non-null   int64  
 3   col04   5214 non-null   object 
 4   col05   5214 non-null   object 
 5   col06   5214 non-null   object 
 6   col07   5214 non-null   object 
 7   col08   5214 non-null   object 
 8   col09   5150 non-null   object 
 9   col10   3591 non-null   object 
 10  col11   5150 non-null   float64
 11  col12   5150 non-null   object 
 12  col13   2914 non-null   object 
 13  col14   312 non-null    object 
 14  col15   5149 non-null   float64
 15  col16   5148 non-null   float64
dtypes: float64(3), int64(2), object(11)
memory usage: 651.9+ KB


A célula indica confirma que o dataset possui 5.214 linhas e 16 colunas.

Em relação aos tipos, observamos que entre as colunas em que eram esperados números inteiros (aquelas correspondentes a temporada, id da partida e placares), as colunas `col11`, `col14`e `col15` estão tipadas como float64, possivelmente por possuírem valores `nan` nos registros faltantes.

E, ainda sobre dados faltantes, vemos que algumas colunas não deveriam possuí-los, como em `col09` (nome da equipe mandante) e `col15`/`col16` (placar de pênaltis, que deveriam possuir par diferente de 0/0 nas partidas com disputa de pênaltis e 0/0 em todos os demais casos). Como, dentre estes casos, a `col16` é a que parece ter maior número de faltantes, podemos começar a investigar a partir dela:

In [5]:
# armazenando as linhas com valor nulo na col16 no dataframe col16_null
col16_null = raw[raw['col16'].isnull()].copy()
col16_null

,col01,col02,col03,col04,col05,col06,col07,col08,col09,col10,col11,col12,col13,col14,col15,col16
1051,campeonato-brasileiro-serie-a,2014,292,Ilha do Retiro - Recife - PE,Quarta,22 de Outubro de 2014,22:30,0,Sport - PE,NaN,1.0,Goiás - GO,Esquerdinha 45' (2ºT),NaN,NaN,NaN
1897,campeonato-brasileiro-serie-a,2016,378,Arena Condá - Chapeco - SC,Domingo,11 de Dezembro de 2016,17:00,Chapecoense - SC,NaN,Atlético - MG,NaN,NaN,0,0,NaN,NaN
3828,copa-brasil-masculino,2012,29,Presidente Vargas/CE - Fortaleza - CE,Quarta,14 de Março de 2012,20:30,Ceará - CE,NaN,Gama - DF,NaN,NaN,0,0,NaN,NaN
3830,copa-brasil-masculino,2012,31,João Lamego - Ipatinga - MG,Quarta,14 de Março de 2012,20:30,Betim - MG,NaN,Real Noroeste Capixaba - ES,NaN,NaN,0,0,NaN,NaN
3832,copa-brasil-masculino,2012,33,Alfredo Jaconi - Caxias do Sul - RS,Quarta,14 de Março de 2012,20:30,Juventude - RS,NaN,Operario Ferroviario Esporte C - PR,NaN,NaN,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4502,copa-brasil-masculino,2016,61,Nabi Abi Chedid - Braganca Paulista - SP,Terça,26 de Abril de 2016,19:15,Bragantino - SP,NaN,Brasília - DF,NaN,NaN,0,0,NaN,NaN
4509,copa-brasil-masculino,2016,68,Couto Pereira - Curitiba - PR,Quarta,27 de Abril de 2016,19:30,Coritiba - PR,NaN,Guarany - CE,NaN,NaN,0,0,NaN,NaN
4545,copa-brasil-masculino,2016,104,Mineirão - Belo Horizonte - MG,Terça,17 de Maio de 2016,21:30,Cruzeiro - MG,NaN,Londrina - PR,NaN,NaN,0,0,NaN,NaN
4547,copa-brasil-masculino,2016,106,Vila Belmiro - Santos - SP,Quarta,18 de Maio de 2016,19:30,Santos - SP,NaN,Galvez - AC,NaN,NaN,0,0,NaN,NaN


Em consulta ao site da CBF pelo nome da competição, edição e id da partida (usando a url **cbf.com.br/futebol-brasileiro/competicoes/`{competicao}`/`{temporada}`/`{id_partida}`?ref=linha** para cada caso), constatamos que:
- O primeiro (linha 1051) refere-se a uma partida cuja página tem formatação levemente diferente no trecho do HTML que identificaria eventual disputa de pênaltis, o que deve ter quebrado a lógica da raspagem e impedido a anotação dos valores `0` nos campos adequados
- O segundo (linha 1897) refere-se a uma partida da época em que ocorreu o acidente aéreo que vitimou os jogadores da Chapecoense - SC. O WO duplo foi acordado com a equipe adversária e a partida não foi realizada (mais detalhes [aqui](https://www.uol.com.br/esporte/futebol/ultimas-noticias/2016/12/05/presidente-da-chape-diz-que-cbf-ja-cancelou-jogo-contra-o-atletico-mg.htm)).
- Os demais correspondem a jogos da Copa do Brasil, de diferentes edições, que não foram realizados por conta da regra que prevê, nas fases iniciais da competição, a dispensa do jogo de volta se a equipe visitante vencer o jogo de ida por dois ou mais gols de diferença.
  - Nestas linhas, é possível observar que as colunas `col09` e `col15` também possuem valores nulos, sugerindo que a remoção destas linhas também deve servir como tratamento para os valores faltantes destas colunas.

Sendo assim, podemos apenas corrigir o primeiro caso e remover os demais do dataset:

**Corrigindo o primeiro caso:**

In [6]:
# corrigindo a linha 1051
raw.at[1051, 'col15'] = 0
raw.at[1051, 'col16'] = 0
raw.iloc[1051]

col01    campeonato-brasileiro-serie-a
col02                             2014
col03                              292
col04     Ilha do Retiro - Recife - PE
col05                           Quarta
col06            22 de Outubro de 2014
col07                            22:30
col08                                0
col09                       Sport - PE
col10                              NaN
col11                              1.0
col12                       Goiás - GO
col13           Esquerdinha 45' (2ºT) 
col14                              NaN
col15                              0.0
col16                              0.0
Name: 1051, dtype: object

<br>

**Removendo os demais:**

In [7]:
# remove a primeira linha do df col16_null
col16_null.drop(index=1051, inplace=True)
# coleta os índices do df col16_null
index_drop = col16_null.index
# dropa do df original as linhas correspondentes aos índices coletados
raw.drop(index=index_drop, inplace=True)
# confere se o df original ainda possui linhas com valor nulo na col16
raw[raw['col16'].isnull()]

,col01,col02,col03,col04,col05,col06,col07,col08,col09,col10,col11,col12,col13,col14,col15,col16


Agora, podemos testas as colunas `col09` e `col15` para verificar se ainda possuem valores nulos:

In [8]:
raw[raw['col09'].isnull()]

,col01,col02,col03,col04,col05,col06,col07,col08,col09,col10,col11,col12,col13,col14,col15,col16


In [9]:
raw[raw['col15'].isnull()]

,col01,col02,col03,col04,col05,col06,col07,col08,col09,col10,col11,col12,col13,col14,col15,col16


## Criando o dataset de dados tratados das partidas

Com estes procedimentos preliminares, conseguimos endereçar os principais casos de valores nulos e podemos iniciar a criação do dataset que deve conter os dados principais das partidas já tratados. Criamos um dataset vazio e começamos a populá-lo com as colunas que não precisam de tratamento (correspondentes ao nome da competição, à temporada e ao id de cada partida):

In [10]:
partidas = pd.DataFrame()
partidas['competicao'] = raw['col01']
partidas['temporada'] = raw['col02']
partidas['id_partida'] = raw['col03']

Para os dados sobre o local da partida, desmembramos a coluna `col04` dos dados originais, que traz o nome do estádio e, na maioria dos casos, também a cidade e o Estado onde se encontra:

In [11]:
local = raw['col04'].str.split(' - ', expand=True)
local.columns = ['estadio', 'cidade', 'uf']
partidas = pd.concat([partidas, local], axis=1)

Antes de incorporar os dias da semana ao dataset tratado, primeiro faremos uma verificação dos valores existentes na coluna: 

In [12]:
raw['col05'].value_counts()

Domingo    1741
Quarta     1684
Sábado      834
Quinta      596
Segunda     147
Terça       124
Sexta        23
Name: col05, dtype: int64

Uma vez confirmada a correção dos valores, podemos convertê-los para o tipo categórico antes de incorporá-los ao dataset tratado, o que tornará mais fácil eventuais análises temporais que demandem a ordem correta dos dias da semana. A esta altura, também podemos incorporar os registros de data/hora das partidas, convertendo-os para os tipos date/time da biblioteca **datetime**:

In [13]:
dias_da_semana = ['Domingo', 'Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado']
partidas['dia_semana'] = pd.Categorical(raw['col05'], categories=dias_da_semana, ordered=True)
partidas['data'] = raw['col06'].apply(lambda d: datetime.strptime(d, ' %d de %B de %Y').date())
partidas['hora'] = raw['col07'].apply(lambda h: datetime.strptime(h, '%H:%M').time())

Chegamos às colunas `col09` e `col12` - aquelas que contêm os nomes das equipes e, provavelmente, oferecem os maiores desafios de tratamento. Vejamos uma amostra dos nomes de equipes existentes no dataset:

In [14]:
equipes = pd.concat([raw['col09'], raw['col12']]).unique()
equipes.sort()
print('Total de nomes distintos:', len(equipes))
print(' * '.join(equipes))

Total de nomes distintos: 279
4 de Julho - PI * 7 de Setembro - MS * A.b.c. - RN * A.s.a. - AL * ABC - RN * Abc - RN * Afogados - PE * Aguia - PA * Aguia Negra - MS * Aimoré - RS * Alecrim - RN * Altos - PI * Amadense - SE * America - MG * America Fc - MG * Americano - RJ * América - MG * América - RN * América de Natal - RN * Anapolina - GO * Anápolis - GO * Aparecidense - GO * Aquidauanense - MS * Aquidauanense Futebol Clube - MS * Aracruz - ES * Arapongas Esporte Clube - PR * Asa - AL * Athletico Paranaense - PR * Atletico - ES * Atletico - PR * Atlético - AC * Atlético - BA * Atlético - GO * Atlético - MG * Atlético - PR * Atlético Cearense - CE * Atlético Mineiro - MG * Atlético Paranaense - PR * Audax - SP * Auto Esporte - PB * Avaí - SC * Avenida - RS * Bahia - BA * Bahia de Feira - BA * Bangu - RJ * Barbalha - CE * Betim - MG * Boa - MG * Boavista - RJ * Boavista Sport Club (antigo Esporte Clube Barreira) - RJ * Botafogo - PB * Botafogo - RJ * Bragantino - PA * Bragantino - SP 

Entre os diversos pontos de atenção nos valores existentes para estas colunas, podemos destacar:
- Nomes abreviados com e sem pontuação. Ex: ***A.b.c* - RN** e ***ABC* - RN**
- Menção aos termos *Esporte Clube* (**Arapongas *Esporte Clube* - PR**), *Futebol Clube* (**Paulista *Futebol Clube* - SP**), *FC* (**América *Fc***) etc., que são comuns à maioria dos nomes das equipes, mas que só aparecem grifados em alguns casos.
  - Ainda, há casos em que o nome da mesma equipe aparece com e sem os termos, como **Santos - SP** e **Santos Fc - SP**
- Nomes semelhantes com diferentes grafias. Ex: **Crici*u*ma - SC** e **Crici*ú*ma - SC**
- Nomes que possuem a referência geográfica da equipe em alguns registro e não em outros. Ex: **América - RN** e **América *de Natal* - RN**

Além destes, há também os casos mais comuns de excesso de espaço antes, entre ou ao final das palavras, capitalização geral dos caracteres, acentuação etc.

Tentaremos endereçar todos estes casos numa única função, a ser aplicada a todos os nomes:

In [15]:
def normaliza_nome(nome):
    nome = nome.strip().upper().replace('.', '')
    nome = re.sub('[EF]\s?C\s', '', nome)
    nome = nome.replace('FUTEBOL CLUBE', '').replace('ESPORTE CLUBE', '')
    com_acento = 'ÄÁÀÃÂËÉÈÊÏÍÌÎÖÓÒÕÔÜÚÙÛÇÑ'
    sem_acento = 'AAAAAEEEEIIIIOOOOOUUUUCN'
    table = nome.maketrans(com_acento, sem_acento)
    nome = nome.translate(table)
    nome = nome.replace('ATHLETICO', 'ATLETICO')
    nome = nome.replace('C R B', 'CRB')
    nome = nome.replace('DE NATAL - RN', '- RN')\
                .replace('PARANAENSE - PR', '- PR')\
                .replace('CEARENSE - CE', '- CE')\
                .replace('MINEIRO - MG', '- MG')\
                .replace('DO PIAUI - PI', '- PI')
    nome = re.sub('\s+', ' ', nome)
    return nome

Agora, aplicamos a função e conferimos uma amostra menor dos nomes das equipes, para verificação dos resultados:

In [16]:
partidas['mandante'] = raw['col09'].apply(normaliza_nome)
partidas['visitante'] = raw['col12'].apply(normaliza_nome)
equipes = pd.concat([partidas['mandante'], partidas['visitante']]).unique()
equipes.sort()
print('Total de nomes distintos:', len(equipes))
'  *  '.join(np.random.choice(equipes, 50))

Total de nomes distintos: 256


'PORTUGUESA - SP  *  NAUTICO - PE  *  JUAZEIRENSE - BA  *  TREM - AP  *  INDEPENDENTE DE TUCURUI - PA  *  TOMBENSE - MG  *  RIVER - PI  *  CORINTHIANS - SP  *  VITORIA - ES  *  IVINHEMA - MS  *  SINOP - MT  *  J MALUCELLI - PR  *  CASCAVEL - PR  *  OESTE - SP  *  BRAGANTINO - SP  *  PARNAHYBA - PI  *  BRUSQUE - SC  *  VILHENA - RO  *  SAO JOSE - RS  *  CIANORTE - PR  *  CAMPINENSE - PB  *  BOAVISTA - RJ  *  RETRO - PE  *  BRAGANTINO - SP  *  PALMEIRAS - SP  *  GREMIO ESPORTIVO SAPUCAIENSE - RS  *  FRIBURGUENSE - RJ  *  MOTO CLUB - MA  *  RIO BRANCO - ES  *  OPERARIO FERROVIARIO ESPORT- PR  *  REAL NOROESTE - ES  *  AMERICA - MG  *  VILHENENSE - RO  *  VASCO DA GAMA - RJ  *  FOZ DO IGUACU - PR  *  VILA NOVA - GO  *  FRIBURGUENSE - RJ  *  CORITIBA - PR  *  J MALUCELLI - PR  *  CENTRAL - PE  *  INTERPORTO - TO  *  DUQUE DE CAXIAS - RJ  *  IVINHEMA - MS  *  VITORIA - ES  *  SFRANCISCO - PA  *  AVAI - SC  *  SANTA CRUZ - PE  *  FORTALEZA - CE  *  VITORIA DA CONQUISTA - BA  *  ATLETICO - MG'

Também podemos ver que o número de nomes distintos foi reduzido de 279 para 256, pois a aplicação da função de padronização identificou variantes dos nomes de algumas equipes.

Agora, acrescentamos ao dataset tratado os placares do tempo normal de jogo e de eventual disputas de pênaltis:

In [17]:
# placar do tempo normal
partidas['mand_placar'] = raw['col08'].apply(lambda g: int(g))
partidas['visit_placar'] = raw['col11'].apply(lambda g: int(g))
# flag para identificação de jogos com disputas de pênaltis
mand_zero_penaltis = raw['col15'].fillna(0).apply(lambda x: x==0)
visit_zero_penaltis = raw['col16'].fillna(0).apply(lambda x: x==0)
zero_penaltis = mand_zero_penaltis * visit_zero_penaltis
raw[~zero_penaltis]
partidas['disputa_penaltis'] = ~zero_penaltis
# placar das disputas de pênaltis
partidas['mand_penaltis'] = raw['col15'].fillna(0).apply(lambda p: int(p))
partidas['visit_penaltis'] = raw['col16'].fillna(0).apply(lambda p: int(p))

Finalmente, podemos conferir o dataset com todos os dados tratados:

In [18]:
partidas

,competicao,temporada,id_partida,estadio,cidade,uf,dia_semana,data,hora,mandante,visitante,mand_placar,visit_placar,disputa_penaltis,mand_penaltis,visit_penaltis
0,campeonato-brasileiro-serie-a,2012,1,São Januário,Rio de Janeiro,RJ,Domingo,2012-05-20,18:30:00,VASCO DA GAMA - RJ,GREMIO - RS,2,1,False,0,0
1,campeonato-brasileiro-serie-a,2012,2,Pituaçu,Salvador,BA,Domingo,2012-05-20,18:30:00,BAHIA - BA,SANTOS - SP,0,0,False,0,0
2,campeonato-brasileiro-serie-a,2012,3,Pacaembu,Sao Paulo,SP,Sábado,2012-05-19,18:30:00,PALMEIRAS - SP,PORTUGUESA - SP,1,1,False,0,0
3,campeonato-brasileiro-serie-a,2012,4,Orlando Scarpelli,Florianopolis,SC,Sábado,2012-05-19,21:00:00,FIGUEIRENSE - SC,NAUTICO - PE,2,1,False,0,0
4,campeonato-brasileiro-serie-a,2012,5,Pacaembu,Sao Paulo,SP,Domingo,2012-05-20,16:00:00,CORINTHIANS - SP,FLUMINENSE - RJ,0,1,False,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5209,copa-brasil-masculino,2021,118,Mineirão,Belo Horizonte,MG,Quarta,2021-10-20,21:30:00,ATLETICO - MG,FORTALEZA - CE,4,0,False,0,0
5210,copa-brasil-masculino,2021,119,Maracanã,Rio de Janeiro,RJ,Quarta,2021-10-27,21:30:00,FLAMENGO - RJ,ATLETICO - PR,0,3,False,0,0
5211,copa-brasil-masculino,2021,120,Arena Castelão,Fortaleza,CE,Quarta,2021-10-27,21:30:00,FORTALEZA - CE,ATLETICO - MG,1,2,False,0,0
5212,copa-brasil-masculino,2021,121,Mineirão,Belo Horizonte,MG,Domingo,2021-12-12,17:30:00,ATLETICO - MG,ATLETICO - PR,4,0,False,0,0


Para persistir os resultados do tratamento, vamos salvar o dataset em arquivo. Teremos uma versão em csv, pela sua versatilidade de uso em outras ferramentas, e outra em parquet, pela possibilidade deste formato incorporar os tipos de dados atribuídos durante o tratamento (o que não é possível com o formato csv).

In [19]:
partidas.to_parquet('dados/partidas.parquet')
partidas.to_csv('dados/partidas.csv', index=False, encoding='utf8')

## Criando o dataset de dados tratados dos gols marcados em cada partida

O dataset anterior agrupa todos os dados de cada partida que podem ser acomodados numa única linha do dataset. São dados de caracterização da partida existentes nas páginas raspadas, como identificador, data, hora, local, equipes e placar final.

No entanto, uma única partida pode ter nenhum, um ou muitos gols, marcados por qualquer das equipes, e cada gol pode ser anotado por quaisquer um dos jogadores que tenham atuado durante o jogo. Daí, a necessidade de serem armazenados num dataset à parte.

Aqui, o dataset também será criado a partir do tratamento dos dados brutos .

Começemos pelas colunas `col10`, `col13`, que trazem coleções de registros de gols marcados a favor da equipe mandante ou da equipe visitante, respectivamente, e também pela coluna `col14`, que identifica eventuais gols contra.

Primeiro, vamos segmentar os valores não nulos desta coluna pelo caracter `|`, que separa os registros de gol de cada coluna:

In [20]:
dados_gols = raw[['col10', 'col13', 'col14']].fillna('').copy()
dados_gols = pd.concat([partidas, dados_gols], axis=1)
for col in dados_gols.columns[-3:]:
    dados_gols[col] = dados_gols[col].apply(lambda x: [g.strip() for g in x.split('|')])

Agora, os valores das colunas `col10` e `col13` compreendem listas cujos tamanhos são iguais aos placares do mandantes e visitantes, respectivamentes (a coluna `col14` possui valores pontuais pois, como dito antes, identifica eventuais gols contra).

In [21]:
def valida_col_gols(x):
    return False if x==[''] else True
col10_gols = [valida_col_gols(x) for x in dados_gols['col10']]
col13_gols = [valida_col_gols(x) for x in dados_gols['col13']]
col14_gols = [valida_col_gols(x) for x in dados_gols['col14']]
mask_gols = [any([a, b, c]) for a, b, c in zip(col10_gols, col13_gols, col14_gols)]
dados_gols[mask_gols][['competicao', 'temporada', 'id_partida', 'mandante', 'mand_placar', 
                         'visit_placar', 'visitante', 'col10', 'col13', 'col14']].sample(5)

,competicao,temporada,id_partida,mandante,mand_placar,visit_placar,visitante,col10,col13,col14
3661,campeonato-brasileiro-serie-a,2021,242,PALMEIRAS - SP,2,4,RED BULL BRAGANTINO - SP,"[Dudu 43' (1ºT), Raphael Veiga 17' (2ºT)]","[Ytalo 12' (1ºT), Cuello 33' (1ºT), Artur 35' ...",[]
1282,campeonato-brasileiro-serie-a,2015,143,ATLETICO - MG,1,0,FIGUEIRENSE - SC,[Lucas Pratto 18' (2ºT)],[],[]
4166,copa-brasil-masculino,2014,69,PORTUGUESA - SP,2,1,POTIGUAR - RN,"[Laercio 34' (1ºT), Vander 19' (2ºT)]",[Reginaldo JR 25' (1ºT)],[]
2023,campeonato-brasileiro-serie-a,2017,124,SAO PAULO - SP,2,2,ATLETICO - GO,"[Lucas Pratto 12' (2ºT), Marcinho 39' (2ºT)]","[Niltinho 20' (2ºT), Everaldo 40' (2ºT)]",[]
2342,campeonato-brasileiro-serie-a,2018,63,AMERICA - MG,1,3,SAO PAULO - SP,[Rafael Moura 11' (1ºT)],"[Diego Souza 8' (1ºT), Nene 45+2' (1ºT), Nene ...",[]


Agora, definimos uma função capaz de receber cada registro existente nas listas das colunas `col10` e `col13` e extrair o nome do jogador e o minuto de jogo correspondente ao gol anotado:

In [22]:
def format_marcacao_gol(jogador_minuto_periodo):
    periodo = jogador_minuto_periodo[-4:-1]
    jogador_minuto = jogador_minuto_periodo[:-7]
    jogador = re.sub(r'[\d]{1,2}(\+[\d]{1,2})?$', '', jogador_minuto).strip()
    minuto = jogador_minuto.replace(jogador, '').strip()
    if minuto.find('+') > 0:
        minuto = 45.5 # atribui o valor 45.5 a qualquer gol anotado nos acréscimos
    minuto = float(minuto)
    if tempo=='2ºT':
        minuto += 45
    return [jogador, minuto]

Uma coisa a ser observada é que a forma como o tempo de jogo é originalmente expresso nos registros (**`8' (1ºT)`**, **`45+3' (1ºT)`** ou **`21' (2ºT)`**, por exemplo) torna um pouco mais difícil sua manipulação como variável ordinal. Por isso, os valores de tempo de jogo foram convertidos para o intervalo entre 1 e 90.5, sendo que o valor 45.5 representa os gols marcados nos acréscimos do primeiro tempo, os valores acima de 45.5 os gols marcados durante o segundo tempo e o valor 90.5 os gols marcados nos acréscimos do segundo tempo.

Podemos, agora, aplicar a função definida anteriormente e coletar os resultados da lógica abaixo na variável `registro_gols`:

In [23]:
registro_gols = []

for idx, row in dados_gols.iterrows():
    if row['col10']!=['']:
        for m in row['col10']:
            registro = [row['competicao'], row['temporada'], row['id_partida'],
                        row['estadio'], 'mandante', row['mandante']]
            lado = 'pro'
            if m in row['col14']:
                lado = 'contra'
            jogador, minuto = format_marcacao_gol(m)
            registro += [lado, jogador, minuto]
            registro_gols.append(registro)
    if row['col13']!=['']:
        for n in row['col13']:
            registro = [row['competicao'], row['temporada'], row['id_partida'],
                        row['estadio'], 'visitante', row['visitante']]
            lado = 'pro'
            if n in row['col14']:
                lado = 'contra'
            jogador, minuto = format_marcacao_gol(n)
            registro += [lado, jogador, minuto]
            registro_gols.append(registro)

NameError: name 'tempo' is not defined

A variável `registro_gols` é uma lista homogênea de listas com o mesmo comprimento. Sua estrutura tabular nos permite popular um dataframe:

In [ ]:
cols = ['competicao', 'temporada', 'id_partida', 'estadio', 'campo', 'equipe', 'pro_contra', 'jogador', 'minuto']
gols = pd.DataFrame(registro_gols, columns=cols)
gols.sort_values(by=['competicao', 'temporada', 'id_partida', 'minuto'], inplace=True)

Assim, temos um dataframe contendo todos os gols extraídos do dataset original:

In [ ]:
gols

Por fim, vamos armazenar em arquivo os dados de gols já tratados. Utilizaremos a mesma estratégia adotada para os dados das partidas, gerando um csv e um parquet:

In [ ]:
gols.to_parquet('dados/gols.parquet')
gols.to_csv('dados/gols.csv', index=False, encoding='utf8')